In [1]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

# API

In [2]:
data = pd.read_csv('../data/final_dataset_to_mongo.csv', index_col='Unnamed: 0')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2347 entries, 0 to 2346
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Brand        2347 non-null   object 
 1   Category     2347 non-null   object 
 2   Name         2347 non-null   object 
 3   Skin_Type    2346 non-null   object 
 4   Ingredients  2347 non-null   object 
 5   URL          2347 non-null   object 
 6   Price        2347 non-null   float64
 7   Rating       2347 non-null   float64
dtypes: float64(2), object(6)
memory usage: 165.0+ KB


## Mongo queries

For example:

In [4]:
con = MongoClient("localhost:27017")

In [5]:
db = con.get_database("skincare")

In [6]:
collection = db.get_collection("first")

In [7]:
# Single query

one = list(collection.find({'Brand':'Anthony'}, {'_id':0}))

In [8]:
one[0]

{'Brand': 'Anthony',
 'Category': 'Cleanser',
 'Name': 'Glycolic Facial Cleanser',
 'Skin_Type': 'Normal, Dry, Combination, Oily ',
 'Ingredients': ' -Glycolic Acid (4.9%): Works to loosen the surface layer of dead cells and helps clear oil.-Aloe Vera: Moisturizes- soothes- and promotes healing while calming skin without creating a greasy feeling.-Vitamin A: Provides antioxidant action to soften- smooth- and increase elasticity of skin.-Vitamin C: Provides antioxidant action to protect- firm- and brighten skin.\n\n\n\nWater- Cocoamidopropyl Oxide- Glycerin- Cetyl Alcohol- Stearyl Alcohol- Potassium Hydroxide- Sodium Phytate- Citrus Grandis Peel Oil- Citrus Noblis Peel Oil- Ascorbic Acid- Tocopheryl Acetate- Retinyl Acetate- Methylchloroisothiazolinone- Methylisothiazolione- Calendula Officinalis Flower Extract- Anthemis Noblis Flower Extract.',
 'URL': 'https://www.sephora.com/product/glycolic-facial-cleanser-P9038?icid2=products grid:p9038',
 'Price': 24.9,
 'Rating': 4.5}

In [9]:
# Query using an input

user_input = input('Please choose a brand:')

Please choose a brand:Clarins


In [10]:
two = list(collection.find({'Brand':user_input}, {'_id':0}).limit(5))

In [11]:
two[0]

{'Brand': 'Clarins',
 'Category': 'Face Serum',
 'Name': 'Double Serum Complete Age Control Concentrate',
 'Skin_Type': 'Normal, Dry, Combination, Oily ',
 'Ingredients': ' -Turmeric: Contains exceptional anti-aging properties.-Oat Sugars: Visibly firm and lift.\n-Orthosiphon Extract: Helps improve the skin texture.-Callicarpa Extract: Revives radiance.Water- Cetearyl Isononanoate- Glycerin- Isononyl Isononanoate- Caprylic/Capric Triglyceride- Pentylene Glycol- Ppg-3 Myristyl Ether- Ethylene/Propylene/Styrene Copolymer. Dipsacus Sylvestris Extract- Butylene Glycol- Parfum/ Fragrance-Phenoxyethanol-Propanediol-Tromethamine. Silybum Marianum Seed Oil- Carbomer-Tocopheryl Acetate- Escin- Cheno -\nPodium Quinoa Seed Extract- Ethylhexylglycerin- Squalane-Avena Sativa (Oat) Kernel Extract- Theobroma Cacao (Cocoa) Extract- Butylene/Ethylene/Styrene Copolymer- Leontopodium Alpinum Extract- Xanthan Gum- Persea Gratissima (Avocado) Oil Unsaponifiables-Caramel-Curcuma Longa (Turmeric) Root Extrac

For API:

In [12]:
# Si me pasan un input  en forma de string separado por comas

inputs = input('Please choose a brand:')

Please choose a brand:Clarins, Anthony


In [13]:
# Lo paso a lista 

inputs_list = inputs.split(',')

In [14]:
inputs_list

['Clarins', ' Anthony']

In [15]:
#  Query: first option

three = list(collection.find({'Brand': {'$in':inputs_list}}, {'_id':0}))

In [16]:
three[0]

{'Brand': 'Clarins',
 'Category': 'Face Serum',
 'Name': 'Double Serum Complete Age Control Concentrate',
 'Skin_Type': 'Normal, Dry, Combination, Oily ',
 'Ingredients': ' -Turmeric: Contains exceptional anti-aging properties.-Oat Sugars: Visibly firm and lift.\n-Orthosiphon Extract: Helps improve the skin texture.-Callicarpa Extract: Revives radiance.Water- Cetearyl Isononanoate- Glycerin- Isononyl Isononanoate- Caprylic/Capric Triglyceride- Pentylene Glycol- Ppg-3 Myristyl Ether- Ethylene/Propylene/Styrene Copolymer. Dipsacus Sylvestris Extract- Butylene Glycol- Parfum/ Fragrance-Phenoxyethanol-Propanediol-Tromethamine. Silybum Marianum Seed Oil- Carbomer-Tocopheryl Acetate- Escin- Cheno -\nPodium Quinoa Seed Extract- Ethylhexylglycerin- Squalane-Avena Sativa (Oat) Kernel Extract- Theobroma Cacao (Cocoa) Extract- Butylene/Ethylene/Styrene Copolymer- Leontopodium Alpinum Extract- Xanthan Gum- Persea Gratissima (Avocado) Oil Unsaponifiables-Caramel-Curcuma Longa (Turmeric) Root Extrac

In [17]:
#  Query: second option

inputs_list_final = []
for inputs in inputs_list:
    inputs_query = list(collection.find({'Brand': {'$regex':  f'.*{inputs}.*'}}, {'_id':0}))
    inputs_list_final.append(inputs_query)

In [18]:
inputs_list_final[1]

[]

In [19]:
# According to skin type

skin_variable = data.Skin_Type[0]

In [20]:
skin_variable_list = skin_variable.split(',')

In [21]:
four = list(collection.find({'Skin_Type': {'$in':skin_variable_list}}, {'_id':0}))

In [22]:
four

[{'Brand': 'Algenist',
  'Category': 'Moisturizer',
  'Name': 'Regenerative Anti-Aging Moisturizer',
  'Skin_Type': 'Normal',
  'Ingredients': ' -Alguronic Acid: Increases cell regeneration and elastin synthesis. -Vitamin C: Boosts radiance and reduces wrinkles. -Apple Cell Extract: Delays cell aging and stimulates cell regeneration and repair.Water/Eau (Aqua)- Caprylic/Capric Triglyceride- Butylene Glycol- Dimethicone- Pentylene Glycol- Butyrospermum Parkii (Shea) Butter- Sorbitan Stearate- Glycerin- Glyceryl Stearate- PEG-100 Stearate- Limnanthes Alba (Meadowfoam) Seed Oil- DI-C12-15 Alkyl Fumarate- Algae Exopolysaccharides- Malus Domestica Fruit (Apple) Cell Culture Extract- Bambusa Vulgaris (Bamboo) Leaf/Stem Extract- Tetrapeptide-21- Pisum Sativum (Pea) Extract- Enantia Chlorantha Bark Extract- Alaria Esculenta Extract- Sodium Ascorbyl Phosphate- Biotin- Glucosamine HCL- Eucalyptus Globulus Leaf Oil- Ergothioneine- Oleanolic Acid- Cetyl Alcohol- Dimethicone/PEG-10/15 Crosspolymer-

In [33]:
# Si me pasan un input  en forma de string separado por comas

inputs = list(input('Please choose a brand:'))

Please choose a brand:Oily


In [46]:
skin_type_list = []
for d in data.Skin_Type:
    skin_type_list.append(str(d).split(','))

In [47]:
skin_type_list

[['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily', ' Sensitive '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal'],
 ['Normal'],
 ['Normal', ' Dry', ' Combination', ' Oily', ' Sensitive '],
 ['Normal'],
 ['Normal', ' Dry', ' Combination', ' Oily', ' Sensitive '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily', ' Sensitive '],
 ['Normal', ' Dry', ' Combination', ' Oily', ' Sensitive '],
 ['Normal'],
 ['Normal', ' Dry', ' Combination', ' Oily '],
 ['Normal', ' Dry', ' Combination', ' Oily', ' Sensitive '],
 ['Normal'],
 ['Normal'],
 ['Normal', ' Dry'],
 ['Normal', ' Dry'

In [55]:
query_list = []
for s in skin_type_list:
    if inputs in s:
        query = list(collection.find({'Skin_Type': inputs}, {'_id':0}))
        query_list.append(query)

In [57]:
query = list(collection.find({'Skin_Type':{"$in": [ 'Combination ', 'Normal']} }, {'_id':0}))
query[0]

{'Brand': 'Algenist',
 'Category': 'Moisturizer',
 'Name': 'Regenerative Anti-Aging Moisturizer',
 'Skin_Type': 'Normal',
 'Ingredients': ' -Alguronic Acid: Increases cell regeneration and elastin synthesis. -Vitamin C: Boosts radiance and reduces wrinkles. -Apple Cell Extract: Delays cell aging and stimulates cell regeneration and repair.Water/Eau (Aqua)- Caprylic/Capric Triglyceride- Butylene Glycol- Dimethicone- Pentylene Glycol- Butyrospermum Parkii (Shea) Butter- Sorbitan Stearate- Glycerin- Glyceryl Stearate- PEG-100 Stearate- Limnanthes Alba (Meadowfoam) Seed Oil- DI-C12-15 Alkyl Fumarate- Algae Exopolysaccharides- Malus Domestica Fruit (Apple) Cell Culture Extract- Bambusa Vulgaris (Bamboo) Leaf/Stem Extract- Tetrapeptide-21- Pisum Sativum (Pea) Extract- Enantia Chlorantha Bark Extract- Alaria Esculenta Extract- Sodium Ascorbyl Phosphate- Biotin- Glucosamine HCL- Eucalyptus Globulus Leaf Oil- Ergothioneine- Oleanolic Acid- Cetyl Alcohol- Dimethicone/PEG-10/15 Crosspolymer- Stea

### Working on API lists

In [ ]:
data.sample(3)

In [ ]:
list_brand = list(data.Brand.unique())

In [ ]:
list_brand

In [ ]:
list_category = list(data.Category.unique())

In [ ]:
list_category

In [ ]:
data.Skin_Type.unique()

In [ ]:
mini = float(data.Price.min())

In [ ]:
mini

In [ ]:
data.Price.min()

In [ ]:
maxi = int(data.Price.max())

In [ ]:
maxi

In [ ]:
data.Price.max()

In [ ]:
data.Rating.max()